<a href="https://colab.research.google.com/github/Rajesh-Khanna/NLP-Drug-review-classification/blob/master/Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Download Dataset: <a href="https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29">link</a></center>

In [3]:
!curl --output dataset.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip dataset.zip
!rm dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.9M  100 40.9M    0     0  12.1M      0  0:00:03  0:00:03 --:--:-- 12.1M
Archive:  dataset.zip
replace drugsComTest_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: drugsComTest_raw.tsv    
replace drugsComTrain_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: drugsComTrain_raw.tsv   


# <center>Imports</center>

In [0]:
import pandas as pd
import numpy as np
import html

# <center>Processing to Tokens</center>

## Read dataset

In [5]:
import pandas as pd
import html

def load_dataset(filename):
    df = pd.read_csv(filename, sep='\t')
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(['drugName', 'condition', 'date', 'usefulCount'], axis=1, inplace=True)
    df['review'] = df['review'].apply(lambda x: x[1:-1])
    df['review'] = df['review'].apply(html.unescape)
    df['rating'] = df['rating'].astype('int64')
    df['label'] = df['rating']
    df.label[df['rating'] <= 4] = 0
    df.label[(df['rating'] <= 6) & (df['rating'] >= 5)] = 1
    df.label[df['rating'] >= 7] = 2
    df.drop(['rating'], axis=1, inplace=True)
    return df

df_train = load_dataset("drugsComTrain_raw.tsv")
df_test = load_dataset("drugsComTest_raw.tsv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Removing Stopwords and Tokenizing the reviews

In [6]:
import nltk
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.corpus import stopwords 

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
def func(row):
    l = tokenizer.tokenize(row['review'])
    s = [i.lower() for i in l if i not in stop_words]
    return s
df_train['processed'] = df_train.apply (lambda row: func(row), axis=1)
df_test['processed'] = df_test.apply (lambda row: func(row), axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df_train[1:6]

,review,label,processed
1,My son is halfway through his fourth week of I...,2,"[my, son, halfway, fourth, week, intuniv, we, ..."
2,"I used to take another oral contraceptive, whi...",1,"[i, used, take, another, oral, contraceptive, ..."
3,This is my first time using any form of birth ...,2,"[this, first, time, using, form, birth, contro..."
4,Suboxone has completely turned my life around....,2,"[suboxone, completely, turned, life, around, i..."
5,2nd day on 5mg started to work with rock hard ...,0,"[2nd, day, 5mg, started, work, rock, hard, ere..."


## Remove very large sentences if needed

In [0]:
df_train['processed_len'] = df_train.processed.apply(len)
df_test['processed_len'] = df_test.processed.apply(len)
df_train = df_train[df_train['processed_len'] <= 100].reset_index(drop=True)
df_test = df_test[df_train['processed_len'] <= 100].reset_index(drop=True)

# <center>Embedding the Tokens</center>

## Glove embedings

### Download Glove embeddings

In [0]:
!wget "http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip"
!unzip glove.6B.zip
!rm glove.6B.zip

--2019-11-01 19:40:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  2.10MB/s    in 6m 29s  

2019-11-01 19:46:31 (2.11 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


### Util functions

In [0]:
def embed(processed, w2v_dict, default_vec):
    vectors = []
    for w in processed:
        if(w in w2v_dict):
            vectors.append(w2v_dict[w])
        else:
            vectors.append(default_vec)
    vectors = np.array(vectors)
    return vectors

def embed_using_glove(df, embed_dim=50):
    # Load Glove embeddings.
    GLOVE_FILE = 'glove.6B.' + str(embed_dim) + 'd.txt'
    # Get number of vectors and hidden dimensions
    with open(GLOVE_FILE, 'r') as f:
        for i, line in enumerate(f):
            pass
    n_vec = i + 1
    hidden_dim = len(line.split(' ')) - 1
    # Find the average of all embeddings to be assigned to tokens not in the embeddings.
    # Create a word to vector dictionary for embedding.
    avg_vec = np.zeros((hidden_dim), dtype=np.float32)
    w2v_dict = {}
    count = 0
    with open(GLOVE_FILE, 'r') as f:
        for line in f:
            w2v_dict[line.split(' ')[:1][0]] = np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)
            avg_vec += np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)
            count+=1
    avg_vec/=count

    df['vectors'] = df.apply(lambda row: embed(row['processed'], w2v_dict, avg_vec), axis=1)


### Convert tokens in to Glove embeddings and save them to file.

In [0]:
embed_using_glove(df_train, 50)
df_train.to_pickle("train_glove_50.pckl")
embed_using_glove(df_test, 50)
df_test.to_pickle("test_glove_50.pckl")


## Word2Vec embeddings

### Download the Google word2vec embeddings

In [0]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM -O word2vec.bin.gz
!gunzip word2vec.bin.gz

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/word2vec.bin.gz
1.65GB [00:10, 162MB/s]


### Load the word2vec embeddings into memory

In [0]:
    from gensim.models import KeyedVectors
    
    w2v_model = KeyedVectors.load_word2vec_format('word2vec.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Util funcitons

In [0]:
def embed(processed, w2v_dict, default_vec):
    vectors = []
    for w in processed:
        if(w in w2v_dict):
            vectors.append(w2v_dict[w])
        else:
            vectors.append(default_vec)
    vectors = np.array(vectors)
    return vectors


### Get smaller versions of train and test dataset

In [0]:
df_train_w2v = df_train.sample(n=int(df_train.shape[0]/6)).reset_index(drop=True)
df_test_w2v = df_test.sample(n=int(df_test.shape[0]/6)).reset_index(drop=True)

### Convert the tokens into Word2Vec embeddings and save them to file

In [0]:
w2v_avg = np.average(w2v_model.vectors, axis=0)
df_train_w2v['vectors'] = df_train_w2v.apply(lambda row: embed(row['processed'], w2v_model, w2v_avg), axis=1)
df_test_w2v['vectors'] = df_test_w2v.apply(lambda row: embed(row['processed'], w2v_model, w2v_avg), axis=1)

In [0]:
df_train_w2v.memory_usage(deep=True)

Index                   128
review             13565982
label                214200
processed          13183544
processed_len        214200
vectors          1602832200
dtype: int64

## Elmo embeddings

### Install allennlp

In [0]:
!pip install allennlp

### Imports

In [0]:
from allennlp.commands.elmo import ElmoEmbedder
import sys

### Util functions

In [0]:
def create_elmo_embeddings(elmo, documents, max_sentences = 1000):
    num_sentences = min(max_sentences, len(documents)) if max_sentences > 0 else len(documents)
    print("\n\n:: Lookup of " + str(num_sentences) + " ELMo representations. This takes a while ::")
    embeddings = []
    tokens = documents['processed'].to_numpy()
    
    documentIdx = 0
    for elmo_embedding in elmo.embed_sentences(tokens):  
        document = documents.iloc[documentIdx]
        # Average the 3 layers returned from ELMo
        avg_elmo_embedding = np.average(elmo_embedding, axis=0)             
        embeddings.append(avg_elmo_embedding)        
        
        # Some progress info
        documentIdx += 1
        percent = 100.0 * documentIdx / num_sentences
        line = '[{0}{1}]'.format('=' * int(percent / 2), ' ' * (50 - int(percent / 2)))
        status = '\r{0:3.0f}%{1} {2:3d}/{3:3d} sentences'
        sys.stdout.write(status.format(percent, line, documentIdx, num_sentences))
        
        if max_sentences > 0 and documentIdx >= max_sentences:
            break
            
    return embeddings


### Get smaller versions of train and test dataset

In [0]:
df_train_elmo = df_train.sample(n=int(df_train.shape[0]/20)).reset_index(drop=True)
df_test_elmo = df_test.sample(n=int(df_test.shape[0]/20)).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


### Get ELMo embeddings

In [0]:
elmo = ElmoEmbedder(cuda_device=0) #Set cuda_device to the ID of your GPU if you have one
df_train_elmo['elmo_embeddings'] = create_elmo_embeddings(elmo, df_train_elmo, -1)
df_test_elmo['elmo_embeddings'] = create_elmo_embeddings(elmo, df_test_elmo, -1)